In [2]:
import settrade_v2

In [6]:
from settrade_v2 import Investor

investor = Investor(
                app_id="QxHlBT7kNFlZt8M6",                                 
                app_secret="GRrzAlrGUgCzR70gU1VPeTYg7fSzU/mmmk2omvyYBLA=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Ameen-D")            

account_info = deri.get_account_info()

account_info                                              

{'creditLine': 10000000.0,
 'excessEquity': 10000000.0,
 'cashBalance': 10000000.0,
 'equity': 10000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 10000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [29]:
import psycopg2

# ข้อมูลการเชื่อมต่อ
host = "localhost"           # หรือ IP เช่น "127.0.0.1"
port = "54322"
database = "candlestick"
user = "postgres"            # ✅ แก้จาก posgres เป็น postgres
password = "asdzxc123"

conn = None
cursor = None

try:
    # เชื่อมต่อ
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # สร้างตาราง
    create_table_query = """
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol TEXT,
        time TIMESTAMP,
        open_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        close_price FLOAT,
        volume BIGINT,
        value FLOAT,
        PRIMARY KEY (symbol, time)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    print("✅ ตาราง 'candlestick_data' สร้างสำเร็จ!")

except Exception as e:
    print("❌ เกิดข้อผิดพลาด:", e)

finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()


✅ ตาราง 'candlestick_data' สร้างสำเร็จ!


In [15]:
import psycopg2


In [41]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
normalized=True,)


In [22]:
import pandas as pd

In [23]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,3345196,1692550800,70.75,71.00,70.25,70.50,9418758,0.000000e+00
1,3345196,1692637200,70.50,71.25,70.50,70.75,15882965,0.000000e+00
2,3345196,1692723600,71.00,72.00,71.00,71.75,17388328,0.000000e+00
3,3345196,1692810000,72.00,72.50,71.75,72.00,21931538,0.000000e+00
4,3345196,1692896400,71.75,72.00,71.25,71.50,17256805,0.000000e+00
...,...,...,...,...,...,...,...,...
495,3345196,1756832400,36.00,37.00,35.75,36.75,18638843,0.000000e+00
496,3345196,1756918800,37.00,38.75,37.00,37.25,34445144,0.000000e+00
497,3345196,1757005200,37.50,38.75,37.00,38.50,30081233,0.000000e+00
498,3345196,1757264400,38.75,39.00,37.75,37.75,31085685,0.000000e+00


In [48]:
import os
import time
import psycopg2
import pandas as pd
from settrade_v2 import Investor
from dotenv import load_dotenv
from datetime import datetime

# ✅ โหลดตัวแปรจาก .env
load_dotenv()

# ✅ ดึงค่าจาก .env
db_config = {
    "host": "localhost",
    "port": "54322",  # ✅ ตรงกับที่คุณตั้งไว้
    "database": "candlestick",
    "user": "postgres",
    "password": "asdzxc123"
}

# ✅ สร้าง Investor Object สำหรับเรียก Market Data
investor = Investor(
                app_id="QxHlBT7kNFlZt8M6",                                 
                app_secret="GRrzAlrGUgCzR70gU1VPeTYg7fSzU/mmmk2omvyYBLA=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Ameen-D")    
market = investor.MarketData()

# ✅ ฟังก์ชันดึงข้อมูล Financial Data และบันทึกลง PostgreSQL
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        
        # ✅ ดึงข้อมูลจาก Settrade API
        res = market.get_quote_symbol(symbol)

        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # ✅ เตรียมข้อมูล
        timestamp = datetime.now()
        open_price = res.get("open", 0.0)
        high_price = res.get("high", 0.0)
        low_price = res.get("low", 0.0)
        close_price = res.get("last", 0.0)
        volume = res.get("totalVolume", 0)
        value = res.get("totalValue", 0.0)

        # ✅ เชื่อมต่อฐานข้อมูลและบันทึกข้อมูล
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cursor:
                insert_query = """
                    INSERT INTO candlestick_data (symbol, time, open_price, high_price, low_price, close_price, volume, value)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    symbol, timestamp, open_price, high_price,
                    low_price, close_price, volume, value
                ))
                conn.commit()
                print(f"✅ ข้อมูลของ {symbol} บันทึกเรียบร้อยแล้ว!")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับ {symbol}: {e}")

# ✅ โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # หน่วงเวลาเพื่อไม่ให้ API ถูก block


🔍 กำลังดึงข้อมูล Financial Data สำหรับ 24CS...
✅ ข้อมูลของ 24CS บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 2S...
✅ ข้อมูลของ 2S บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 3BBIF...
✅ ข้อมูลของ 3BBIF บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A...
✅ ข้อมูลของ A บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A5...
✅ ข้อมูลของ A5 บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAI...
✅ ข้อมูลของ AAI บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAV...
✅ ข้อมูลของ AAV บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ABM...
✅ ข้อมูลของ ABM บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACAP...
✅ ข้อมูลของ ACAP บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACC...
✅ ข้อมูลของ ACC บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACE...
✅ ข้อมูลของ ACE บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACG...
✅ ข้อมูลของ ACG บันทึกเรียบร้อยแล้ว!
🔍 กำ